In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import re
from io import StringIO

In [20]:
urls = open('./F1_URLS.txt')
URLS = urls.read().split("\n")

In [23]:
t= URLS[0]

response = requests.get(t+"practice-1.html")
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find_all('table')
df =pd.read_html(StringIO(str(table)))[0]
    
response = requests.get(t+"practice-2.html")
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find_all('table')
df2 =pd.read_html(StringIO(str(table)))[0]
    
response = requests.get(t+"practice-3.html")
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find_all('table')
df3 =pd.read_html(StringIO(str(table)))[0]
    
df.at[0, 'Gap']='+0.0s'
df2.at[0, 'Gap']='+0.0s'
df3.at[0, 'Gap']='+0.0s'
    
response = requests.get(t+"qualifying.html")
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find_all('table')
dfQ =pd.read_html(StringIO(str(table)))[0]
    
df=df.drop(columns=["Unnamed: 0", "Unnamed: 8"])
df["Year"]=re.findall(r'\d+',soup.title.text )[-1]
df["Race"]=(soup.title.text.replace('\n','').replace(' ', '').split('1'))[1].split('GRAND')[0]
    
    
df = df.merge(df2[['No','Pos','Time', 'Gap', 'Laps']], how='inner', on='No', suffixes=(None, '_practice_2'))
df =df.merge(df3[['No','Pos','Time', 'Gap', 'Laps']], how='inner', on='No', suffixes=(None, '_practice_3'))
df = df.merge(dfQ[['No','Pos','Q1', 'Q2', 'Q3', 'Laps']], how='inner', on='No', suffixes=(None, '_qual'))

for t in tqdm(URLS[1:]):
    response = requests.get(t+"practice-1.html")
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find_all('table')
    df1 =pd.read_html(StringIO(str(table)))[0]
    
    response = requests.get(t+"practice-2.html")
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find_all('table')
    df2 =pd.read_html(StringIO(str(table)))[0]
    
    response = requests.get(t+"practice-3.html")
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find_all('table')
    df3 =pd.read_html(StringIO(str(table)))[0]
    
    df1.at[0, 'Gap']='+0.0s'
    df2.at[0, 'Gap']='+0.0s'
    df3.at[0, 'Gap']='+0.0s'
    
    response = requests.get(t+"qualifying.html")
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find_all('table')
    dfQ =pd.read_html(StringIO(str(table)))[0]
    
    df1=df1.drop(columns=["Unnamed: 0", "Unnamed: 8"])
    df1["Year"]=re.findall(r'\d+',soup.title.text )[-1]
    df1["Race"]=(soup.title.text.replace('\n','').replace(' ', '').split('LA1'))[1].split('GRAND')[0]
    
    
    df1 = df1.merge(df2[['No','Pos','Time', 'Gap', 'Laps']], how='inner', on='No', suffixes=(None, '_practice_2'))
    df1 =df1.merge(df3[['No','Pos','Time', 'Gap', 'Laps']], how='inner', on='No', suffixes=(None, '_practice_3'))
    df1 = df1.merge(dfQ[['No','Pos','Q1', 'Q2', 'Q3', 'Laps']], how='inner', on='No', suffixes=(None, '_qual'))
    
    df=pd.concat((df,df1), axis=0)

100%|████████████████████████████████████████████████████████████████████████████████| 149/149 [17:48<00:00,  7.17s/it]


In [24]:
df

,Pos,No,Driver,Car,Time,Gap,Laps,Year,Race,Pos_practice_2,...,Laps_practice_2,Pos_practice_3,Time_practice_3,Gap_practice_3,Laps_practice_3,Pos_qual,Q1,Q2,Q3,Laps_qual
0,1,11,Sergio Perez PER,Red Bull Racing Honda RBPT,1:32.758,+0.0s,21,2023,GULFAIRBAHRAIN,3,...,26,3,1:32.446,+0.106s,12,2,1:31.479,1:30.746,1:29.846,15
1,2,14,Fernando Alonso ALO,Aston Martin Aramco Mercedes,1:33.196,+0.438s,22,2023,GULFAIRBAHRAIN,1,...,25,1,1:32.340,+0.0s,13,5,1:31.158,1:30.645,1:30.336,15
2,3,1,Max Verstappen VER,Red Bull Racing Honda RBPT,1:33.375,+0.617s,21,2023,GULFAIRBAHRAIN,2,...,24,2,1:32.345,+0.005s,13,1,1:31.295,1:30.503,1:29.708,15
3,4,4,Lando Norris NOR,McLaren Mercedes,1:34.165,+1.407s,21,2023,GULFAIRBAHRAIN,9,...,27,13,1:33.202,+0.862s,16,11,1:31.652,1:31.381,NaN,13
4,5,16,Charles Leclerc LEC,Ferrari,1:34.257,+1.499s,17,2023,GULFAIRBAHRAIN,4,...,26,5,1:32.624,+0.284s,21,3,1:31.094,1:30.282,1:30.000,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,17,20,Kevin Magnussen MAG,Renault,1:46.372,+3.503s,21,1,ETIHADAIRWAYSABUDHABI,20,...,25,16,1:43.057,+2.282s,17,18,1:42.142,NaN,NaN,6
16,18,14,Fernando Alonso ALO,McLaren Honda,1:46.379,+3.510s,21,1,ETIHADAIRWAYSABUDHABI,11,...,33,11,1:42.585,+1.810s,14,9,1:41.616,1:41.044,1:41.106,17
17,19,94,Pascal Wehrlein WEH,MRT Mercedes,1:46.458,+3.589s,29,1,ETIHADAIRWAYSABUDHABI,17,...,33,17,1:43.145,+2.370s,17,16,1:41.886,1:41.995,NaN,14
18,20,22,Jenson Button BUT,McLaren Honda,1:47.127,+4.258s,10,1,ETIHADAIRWAYSABUDHABI,12,...,24,13,1:42.664,+1.889s,15,12,1:41.158,1:41.272,NaN,14


In [25]:
df.to_csv('./2016to2023F1PracticeAndQual.csv', index=False)